In [ ]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [ ]:
# pandas series methods
def multiply_func(a : pd.Series, b: pd.Series) -> pd.Series:
    return a * b

x = pd.Series([1,2,3])
print(multiply_func(x,x))

In [ ]:
import pyspark.pandas as ps
# Create a Spark DataFrame

sdf = ps.DataFrame(x, columns=["x"])
sdf

### Iterator of Series UDF

In [ ]:
import pandas as pd
from typing import Iterator
from pyspark.sql.functions import col, pandas_udf, struct

In [ ]:
pdf = pd.DataFrame([1,2,3], columns=["x"])
sdf = ps.DataFrame(pdf)
sdf = sdf.to_spark()
sdf

In [ ]:
@pandas_udf("long")
def plus_one(batch_iter : Iterator[pd.Series]) -> Iterator[pd.Series]:
    for x in batch_iter:
        yield x+1

        

In [ ]:
sdf.select(plus_one(col("x"))).show()

### Iterator of Multiple Series

from typing import Iterator,Tuple

@pandas_udf("long")
def multiply_two_cols(iterator : Iterator[Tuple[pd.Series,pd.Series]]) -> Iterator[pd.Series]:
    for a,b in iterator:
        yield a * b

In [ ]:
sdf.select(multiply_two_cols("x","x")).show()

### Series to  scaler UDF

from pyspark.sql import Window

In [ ]:
spdf = ps.DataFrame({
    "id" : [1,1,1,2,2],
    "v" : [1.0,2.0,3.0,5.0,10.0]
})

sdf = spdf.to_spark()
sdf.show()

In [ ]:
@pandas_udf("double")
def mean_udf(v : pd.Series) -> float:
    return v.mean()

In [ ]:
sdf.select(mean_udf(sdf['v'])).show()

In [ ]:
sdf.groupby("id").agg(mean_udf(sdf["v"])).show()

In [ ]:
w = Window.partitionBy("id").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
sdf.withColumn("mean_v", mean_udf(sdf["v"]).over(w)).show()